# MYMail_review – Notas / comandos rápidos

Este notebook sirve como recordatorio de comandos para:
- Subir nuevos ficheros a **entrada** (CosmosDB)
- Gestión de **usuarios** (CosmosDB)
- Despliegue de imagen (ACR) + restart App Service


## 0) CosmosDB (setup + migración)

Variables mínimas en `.env` / App Service Configuration:
- (Cosmos-only) no se usa `STORAGE_BACKEND`.
- `COSMOS_ENDPOINT=<URI>`
- `COSMOS_KEY=<KEY>`
- `COSMOS_DATABASE=mymailreview`

Crear DB/containers (una vez):


Test de conexión a CosmosDB:


In [ ]:
!python Testing/test_cosmos.py


## 1) Subir un nuevo Excel a ENTRADA (CosmosDB)

Carga el Excel y crea registros en el contenedor `entrada` de CosmosDB.

Nota: si un `record_json` supera el límite seguro, se guarda el payload en Blob (legacy) y en Cosmos queda una referencia.


In [ ]:
# Ya no se usa Azure Blob/Table Storage. La ingesta de entrada ya está migrada a CosmosDB.


In [ ]:
# Si necesitas reingestar, crea un script Cosmos-first o reutiliza tu pipeline de carga.


In [ ]:
# (Deprecated) load_entrada.py eliminado al pasar a Cosmos-only.


Flags útiles:
- `--replace` borra la partición antes de cargar
- `--partition` PartitionKey destino (por defecto: `default`)


## 2) Gestión de usuarios (CosmosDB)

### 2.1 Crear/actualizar usuario


In [ ]:
!python scripts/manage_users.py add --username "usuario1" --password "MiPassword" --role revisor

In [ ]:
!python scripts/manage_users.py add --username "admin" --password "MiPassword" --role admin

### 2.2 Cambiar rol


In [ ]:
!python scripts/manage_users.py set-role --username "usuario1" --role admin

### 2.3 Actualizar contraseña


In [ ]:
!python scripts/manage_users.py set-password --username "usuario1" --password "NuevaPassword"

### 2.4 Listar usuarios


In [ ]:
!python scripts/manage_users.py list

## Requisitos

- Variables en `.env` / `config.py` (CosmosDB: `COSMOS_ENDPOINT`, `COSMOS_KEY`).
- CosmosDB: `COSMOS_ENDPOINT`, `COSMOS_KEY`.
- Dependencias instaladas: `pip install -r requirements.txt`


## 3) Test de Azure OpenAI (legacy)

Usa el script `Testing/test_azure_openai.py` para validar que el endpoint + deployment + api-version funcionan.


In [ ]:
! .\\.venv\\Scripts\\python.exe Testing/test_azure_openai.py --prompt "Responde exactamente: OK" --temperature 1 --max-tokens 256

## 4) Desplegar en cloud (ACR)

Comandos para construir, etiquetar y publicar la imagen en Azure Container Registry.

Antes de empezar:
- Asegúrate de que **Docker Desktop** está iniciado (si no, `docker build` fallará).
- Asegúrate de estar logueado en **Azure**.

Configurar App Service con CosmosDB (placeholders):


In [ ]:
!az webapp config appsettings set --resource-group <RG> --name <APP_NAME> --settings COSMOS_ENDPOINT=<URI> COSMOS_KEY=<KEY> COSMOS_DATABASE=mymailreview


In [ ]:
!docker info


In [ ]:
!az login


In [ ]:
!az acr login -n mymailrevieweuweconreg002


In [20]:
!docker build -t mymailreview .
!docker tag mymailreview mymailrevieweuweconreg002.azurecr.io/mymailreview:latest
!docker push mymailrevieweuweconreg002.azurecr.io/mymailreview:latest


#0 building with "desktop-linux" instance using docker-container driver

#1 [internal] booting buildkit
#1 pulling image moby/buildkit:buildx-stable-1
#1 pulling image moby/buildkit:buildx-stable-1 1.9s done
#1 ERROR: request returned 500 Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/v1.48/images/create?fromImage=moby%2Fbuildkit&tag=buildx-stable-1, check if the server supports the requested API version
------
 > [internal] booting buildkit:
------
ERROR: request returned 500 Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/v1.48/images/create?fromImage=moby%2Fbuildkit&tag=buildx-stable-1, check if the server supports the requested API version
request returned 500 Internal Server Error for API route and version http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/v1.48/images/mymailreview/tag?repo=mymailrevieweuweconreg002.azurecr.io%2Fmymailreview&tag=latest, check if the server supports the

Reiniciar App Service para que cargue el nuevo contenedor (ajusta `<RG>` y `<APP_NAME>`):


In [ ]:
!az webapp restart --resource-group <RG> --name <APP_NAME>


## OLD (legacy) – Blob / Azure Table Storage

La app antigua usaba Azure **Table Storage** para persistencia y Azure **Blob** para entrada/revisiones.

Variables legacy:
- (Deprecated) Table Storage eliminado.
- `COSMOS_ENDPOINT=...`
- `COSMOS_KEY=...`
- `TABLE_USERS/TABLE_LOGS/TABLE_RESULTADOS/TABLE_DESCARTES/TABLE_ENTRADA`
- `AZURE_BLOB_CONTAINER_ENTRADA=entrada`
- `AZURE_BLOB_CONTAINER_REVISIONES=revisiones`

Notas:
- Aunque uses CosmosDB, el Blob se puede seguir usando para `revisiones` y para payloads grandes de `entrada`.
